In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 55.5 MB/s eta 0:00:00


In [ ]:
import mysql.connector
conn=mysql.connector.connect(
    host='18.136.157.135',
    user='dm_team8',
    password='DM!$!Team!82@04!23&',
    database='project_job_market_analysis'
)

In [ ]:
query="SELECT * FROM Market"
df=pd.read_sql(query,conn)
conn.close()

/tmp/ipython-input-3858229869.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,conn)


In [ ]:
print(df.shape)
print(df.info())
print(df.head())
print(df.isnull().sum)

(742, 42)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  742 non-null    int64  
 1   Job_Title           742 non-null    object 
 2   Salary_Estimate     742 non-null    object 
 3   FIELD4              0 non-null      object 
 4   Rating              742 non-null    float64
 5   Company_Name        742 non-null    object 
 6   Location            742 non-null    object 
 7   Headquarters        742 non-null    object 
 8   Size                742 non-null    object 
 9   Founded             742 non-null    int64  
 10  Type_of_ownership   742 non-null    object 
 11  Industry            742 non-null    object 
 12  Sector              742 non-null    object 
 13  Revenue             742 non-null    object 
 14  Competitors         742 non-null    object 
 15  Hourly              742 non-null    int64  
 16

In [ ]:
# Initital Exploration
df.replace(-1,np.nan,inplace=True)

In [ ]:
# Data cleaning
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Columns with missing values and their data types:")
for column in missing_values.index:
    print(f"- {column}: {df[column].dtype}")

for column in missing_values.index:
    if df[column].dtype == 'object':
        mode_value = df[column].mode()[0]
        df[column].fillna(mode_value, inplace=True)
    elif df[column].dtype in ['int64', 'float64']:
        median_value = df[column].median()
        df[column].fillna(median_value, inplace=True)

print("\nMissing values after imputation:")
print(df.isnull().sum().sum())

Columns with missing values and their data types:
- Rating: float64
- Founded: float64
- Age: float64

Missing values after imputation:
0


/tmp/ipython-input-2379826237.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(median_value, inplace=True)


In [ ]:
df['Job_Title'] = df['Job_Title'].str.lower().str.strip()
job_title_mapping = {
    'data scientist i': 'data scientist',
    'data scientist ii': 'data scientist',
    'data scientist iii': 'data scientist',
    'senior data scientist': 'data scientist',
    'sr. data scientist': 'data scientist',
    'lead data scientist': 'data scientist',
    'principal data scientist': 'data scientist',
    'data engineer i': 'data engineer',
    'data engineer ii': 'data engineer',
    'data engineer iii': 'data engineer',
    'senior data engineer': 'data engineer',
    'sr. data engineer': 'data engineer',
    'lead data engineer': 'data engineer',
    'principal data engineer': 'data engineer',
    'data analyst i': 'data analyst',
    'data analyst ii': 'data analyst',
    'data analyst iii': 'data analyst',
    'senior data analyst': 'data analyst',
    'sr. data analyst': 'data analyst',
    'lead data analyst': 'data analyst',
    'principal data analyst': 'data analyst',
    'machine learning engineer': 'machine learning engineer',
    'ml engineer': 'machine learning engineer',
    'research scientist': 'research scientist',
    'research scientist – security and privacy': 'research scientist',
    'sr scientist, immuno-oncology - oncology': 'research scientist',
    'project scientist - auton lab, robotics institute': 'research scientist',
    'analytics engineer': 'analytics engineer',
    'business intelligence analyst': 'business intelligence analyst',
    'bi analyst': 'business intelligence analyst',
    'big data engineer': 'data engineer',
    'cloud engineer': 'cloud engineer',
    'software engineer': 'software engineer',
    'software engineer lll': 'software engineer',
    'software engineer iv': 'software engineer',
    'software engineer v': 'software engineer',
    'director of data science': 'data science manager',
    'data science manager': 'data science manager',
    'manager, data engineering': 'data engineer manager',
    'data engineering manager': 'data engineer manager',
    'director of analytics': 'analytics manager',
    'analytics manager': 'analytics manager',
    'director': 'manager',
    'manager': 'manager',
    'principal': 'manager',
    'lead': 'manager',
    'sr': 'manager',
    'senior': 'manager'

}
df['Job_Title'] = df['Job_Title'].replace(job_title_mapping, regex=True)
display(df['Job_Title'].value_counts().head())

,count
Job_Title,
data scientist,179
data engineer,79
data analyst,32
research scientist,11
marketing data analyst,6


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['Location'] = df['Location'].str.extract(r', ([A-Z]{2})').fillna(df['Location'])
display(df['Location'].value_counts())

,count
Location,
"albuquerque, nm",742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['Location'] = df['Location'].str.extract(r'\, ([a-z]{2})').fillna(df['Location'])
display(df['Location'].value_counts())

,count
Location,
nm,742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['Location'] = df['Location'].str.extract(r'([a-z]{2})$').fillna(df['Location'])
display(df['Location'].value_counts())

,count
Location,
nm,742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['Location'] = df['Location'].str.extract(r',\s*([a-z]{2})$')
df['Location'] = df['Location'].fillna('unknown')
display(df['Location'].value_counts())

,count
Location,
unknown,742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['Location'] = df['Location'].str.extract(r'([a-z]{2})')
display(df['Location'].value_counts())

,count
Location,
un,742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['Location'] = df['Location'].str.split(',').str[-1].str.strip()
display(df['Location'].value_counts())

,count
Location,
un,742


In [ ]:
display(df['Location'].unique())

array(['un'], dtype=object)

In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['State'] = df['Location'].str.extract(r',\s*([a-z]{2})$')
df['Location'] = df['State'].fillna(df['Location'])
df.drop('State', axis=1, inplace=True)
display(df['Location'].value_counts())

,count
Location,
un,742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['State'] = df['Location'].str.extract(r',\s*([a-z]{2})$')
df['Location'] = df['State'].fillna(df['Location'])
df.drop('State', axis=1, inplace=True)
display(df['Location'].value_counts())

,count
Location,
un,742


In [ ]:
df['Location'] = df['Location'].str.lower().str.strip()
df['State'] = df['Location'].str.extract(r',\s*([a-z]{2})$')
df['Location'] = df['State'].fillna(df['Location'])
df.drop('State', axis=1, inplace=True)
display(df['Location'].value_counts())

,count
Location,
un,742


In [ ]:
conn=mysql.connector.connect(
    host='18.136.157.135',
    user='dm_team8',
    password='DM!$!Team!82@04!23&',
    database='project_job_market_analysis'
)
query="SELECT * FROM Market"
df=pd.read_sql(query,conn)
conn.close()

df['Location'] = df['Location'].str.lower().str.strip()
df['State'] = df['Location'].str.extract(r',\s*([a-z]{2})$')
df['Location'] = df['State'].fillna(df['Location'])
df.drop('State', axis=1, inplace=True)
display(df['Location'].value_counts())

/tmp/ipython-input-17122034.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,conn)


,count
Location,
ca,152
ma,103
ny,72
va,41
il,40
md,35
pa,33
tx,28
wa,21


In [ ]:
print(df.shape)
print(df.isnull().sum().sum())
print(df.duplicated().sum())

(742, 42)
742
0


In [ ]:
print("Shape of the DataFrame:", df.shape)
print("Total number of missing values:", df.isnull().sum().sum())
print("Total number of duplicate rows:", df.duplicated().sum())

missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("\nColumns with remaining missing values and their data types:")
for column in missing_values.index:
    print(f"- {column}: {df[column].dtype}")

Shape of the DataFrame: (742, 42)
Total number of missing values: 742
Total number of duplicate rows: 0

Columns with remaining missing values and their data types:
- FIELD4: object


In [ ]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

for column in missing_values.index:
    if df[column].dtype == 'object':
        # Check if the column is empty before calculating mode
        if not df[column].isnull().all():
            mode_value = df[column].mode()[0]
            df[column].fillna(mode_value, inplace=True)
    elif df[column].dtype in ['int64', 'float64']:
        median_value = df[column].median()
        df[column].fillna(median_value, inplace=True)

print("\nTotal number of missing values after imputation:", df.isnull().sum().sum())


Total number of missing values after imputation: 0


In [ ]:
# Task
# States with most jobs
df['state'] = df['Location'].str.split(',').str[-1].str.strip()
state_counts = df['state'].value_counts()
print(state_counts.head(10))

state
ca    152
ma    103
ny     72
va     41
il     40
md     35
pa     33
tx     28
wa     21
nc     21
Name: count, dtype: int64


In [ ]:
#Avg min & max salary by state
df['Avg_Salary'] = (df['Lower_Salary'] + df['Upper_Salary']) / 2
salary_stats = df.groupby('state')['Avg_Salary'].agg(['mean', 'min', 'max'])
print(salary_stats)

             mean    min    max
state                          
al      57.937500   39.5   68.0
az      85.666667   55.0  124.5
ca     123.513158   47.5  232.5
co      86.181818   29.5  133.5
ct      71.100000   62.5   84.0
dc     110.181818   50.0  237.5
de      56.500000   56.5   56.5
fl      77.625000   37.5  132.5
ga      80.666667   79.5   85.0
ia      78.900000   43.0  115.5
id      56.250000   49.0   63.5
il     116.662500   52.5  254.0
in      90.300000   51.5  122.5
ks      87.000000   87.0   87.0
ky      97.000000   84.0  103.5
la      57.625000   41.5   70.5
ma     107.495146   47.5  172.0
md      97.357143   62.0  147.5
mi     100.250000   87.5  130.0
mn      85.500000   76.0   95.0
mo      84.722222   40.5  143.5
nc      98.452381   65.5  134.0
ne      54.250000   51.5   61.5
nj     104.558824   56.5  173.0
nm      74.333333   72.0   75.5
ny      98.652778   15.5  172.0
oh      87.571429   45.5  110.5
or      92.125000   73.0   99.5
pa      93.863636   42.0  155.0
ri     1

In [ ]:
# Average salary in different states
Avg_Salary_by_State = df.groupby('state')['Avg_Salary'].mean()
print(Avg_Salary_by_State)

state
al     57.937500
az     85.666667
ca    123.513158
co     86.181818
ct     71.100000
dc    110.181818
de     56.500000
fl     77.625000
ga     80.666667
ia     78.900000
id     56.250000
il    116.662500
in     90.300000
ks     87.000000
ky     97.000000
la     57.625000
ma    107.495146
md     97.357143
mi    100.250000
mn     85.500000
mo     84.722222
nc     98.452381
ne     54.250000
nj    104.558824
nm     74.333333
ny     98.652778
oh     87.571429
or     92.125000
pa     93.863636
ri    100.000000
sc     60.500000
tn     89.269231
tx     92.678571
ut     94.150000
va     95.621951
wa     93.190476
wi     73.300000
Name: Avg_Salary, dtype: float64


In [ ]:
# Top 5 industries (data science jobs)
data_science_jobs = df[df['Job_Title'].str.contains('data scientist', case=False)]
top_industries = data_science_jobs['Industry'].value_counts().head(5)
print(top_industries)

Industry
Biotech & Pharmaceuticals                  34
Enterprise Software & Network Solutions    25
IT Services                                22
Computer Hardware & Software               21
Insurance Carriers                         20
Name: count, dtype: int64


In [ ]:
# Companies with maximum openings
company_openings = df['Company_Name'].value_counts().head(5)
print(company_openings)

Company_Name
MassMutual\n3.6                        14
Takeda Pharmaceuticals\n3.7            14
Reynolds American\n3.1                 14
Software Engineering Institute\n2.6    11
PNNL\n3.8                              10
Name: count, dtype: int64


In [ ]:
# Jop titles with most jobs
top_titles = df['Job_Title'].value_counts().head(5)
print(top_titles)

Job_Title
Data Scientist           131
Data Engineer             53
Senior Data Scientist     34
Data Analyst              15
Senior Data Engineer      14
Name: count, dtype: int64


In [ ]:
# Salary of top job titles
top_titles_salary = df.groupby('Job_Title')['Avg_Salary'].mean()
print(top_titles_salary)

Job_Title
Ag Data Scientist                                      80.5
Analytics - Business Assurance Data Analyst            43.0
Analytics Consultant                                   66.5
Analytics Manager                                      87.5
Analytics Manager - Data Mart                          64.0
                                                      ...  
Systems Engineer II - Data Analyst                     62.5
Technology-Minded, Data Professional Opportunities     70.5
VP, Data Science                                      124.5
Web Data Analyst                                      106.0
sql_ Data Engineer                                     93.0
Name: Avg_Salary, Length: 264, dtype: float64


In [ ]:
# Skills required by companies
skill_columns = ['Python', 'spark', 'aws', 'excel', 'sql_', 'sas', 'keras', 'pytorch', 'scikit', 'tensor', 'hadoop', 'tableau', 'bi', 'flink', 'mongo', 'google_an']

def get_skills(row):
    skills = [col for col in skill_columns if row[col] == 1]
    return ', '.join(skills) if skills else 'No specific skills listed'

df['Required_Skills'] = df.apply(get_skills, axis=1)

skills_by_company = df.groupby('Company_Name')['Required_Skills'].apply(lambda x: ', '.join(x.unique())).reset_index()
print(skills_by_company)

                      Company_Name  \
0     1-800-FLOWERS.COM, Inc.\n2.8   
1                    1904labs\n4.7   
2                     23andMe\n4.0   
3                  7Park Data\n3.8   
4                    <intent>\n4.6   
..                             ...   
338  e-IT Professionals Corp.\n3.5   
339                     goTRG\n4.2   
340                    h2o.ai\n4.3   
341                    iSeatz\n3.5   
342                 webfx.com\n4.7   

                                       Required_Skills  
0                    Python, excel, sql_, sas, tableau  
1           Python, spark, sql_, keras, scikit, tensor  
2                                           aws, excel  
3        Python, spark, keras, pytorch, scikit, tensor  
4                                        Python, spark  
..                                                 ...  
338                                       Python, sql_  
339                 Python, spark, excel, sql_, hadoop  
340  Python, spark, aws, excel

In [ ]:
# Relation between eduction & salary
education_salary = df.groupby('Degree')['Avg_Salary'].mean()
print(education_salary)

Degree
M     105.634921
P     115.981308
na     94.703655
Name: Avg_Salary, dtype: float64


In [ ]:
df.to_csv('cleaned_data.csv', index=False)